In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Please move to IV to continue training

# I. Install requirements and import library


In [2]:
!pip install tensorboardX
!pip install webcolors
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 30.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import sys
import splitfolders
import shutil
import json

# II. Split training, val, test set and convert PASCAL VOC format into COCO format.

clone tool for convert pascal voc into coco format

cài đặt và import thư viện

In [4]:
# Download source code.
if "voc2coco" not in os.getcwd():
  !git clone --depth 1 https://github.com/yukkyo/voc2coco
  os.chdir('/content/voc2coco')
  sys.path.append('.')
else:
  !git pull

Cloning into 'voc2coco'...
remote: Enumerating objects: 381, done.
remote: Counting objects: 100% (381/381), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 381 (delta 359), reused 376 (delta 359), pack-reused 0
Receiving objects: 100% (381/381), 127.61 KiB | 21.27 MiB/s, done.
Resolving deltas: 100% (359/359), done.


## 1.Download and split data

download data

In [5]:
# download and unzip dataset
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!tar xf VOCtrainval_11-May-2012.tar


--2022-12-30 15:45:05--  http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1999639040 (1.9G) [application/x-tar]
Saving to: ‘VOCtrainval_11-May-2012.tar’

VOCtrainval_11-May- 100%[===================>]   1.86G  11.8MB/s    in 2m 43s  

2022-12-30 15:47:49 (11.7 MB/s) - ‘VOCtrainval_11-May-2012.tar’ saved [1999639040/1999639040]



split data into 3 part: training set, validation set and test set

In [6]:
splitfolders.ratio('/content/voc2coco/VOCdevkit/VOC2012', output="dataset", seed=42, ratio=(.6, .2, .2)) 

Copying files: 40076 files [00:11, 3629.32 files/s]


## 2. Convert PASCAL VOC format into COCO format.


To use tools , we need to reformat the structure to use tools, see detail in here: https://github.com/yukkyo/voc2coco

In [7]:
!rm -rf /content/voc2coco/sample/Annotations
!mkdir /content/voc2coco/sample/Annotations

move file and write .txt file

In [8]:
path = '/content/voc2coco/VOCdevkit/VOC2012/Annotations'
for filename in os.listdir(path):
    fullname = os.path.join(path, filename)
    shutil.move(fullname, '/content/voc2coco/sample/Annotations')  

In [9]:
with open('/content/voc2coco/sample/dataset_ids/train.txt','w') as f:
    list_train = [i[:-4] for i in os.listdir('/content/voc2coco/dataset/train/Annotations')]
    for i in list_train:
        f.write(i)
        f.write('\n')

with open('/content/voc2coco/sample/dataset_ids/val.txt','w') as f:
    list_train = [i[:-4] for i in os.listdir('/content/voc2coco/dataset/val/Annotations')]
    for i in list_train:
        f.write(i)
        f.write('\n')

with open('/content/voc2coco/sample/dataset_ids/test.txt','w') as f:
    list_train = [i[:-4] for i in os.listdir('/content/voc2coco/dataset/test/Annotations')]
    for i in list_train:
        f.write(i)
        f.write('\n')

with open('/content/voc2coco/sample/annpaths_list.txt','w') as f:
    for i in os.listdir('/content/voc2coco/sample/Annotations'):
        f.write('./sample/Annotations/' + i)
        f.write('\n')

with open('/content/voc2coco/sample/annpaths_list.txt','w') as f:
    for i in os.listdir('/content/voc2coco/sample/Annotations'):
        f.write('./sample/Annotations/' + i)
        f.write('\n')

write label to labets.txt

In [10]:
object_list = ['aeroplane','bicycle','bird','boat',\
               'bottle','bus','car','cat','chair','cow','diningtable',\
               'dog','horse','motorbike','person'\
               ,'pottedplant','sheep','sofa','train','tvmonitor']


with open('/content/voc2coco/sample/labels.txt','w') as f:
    for i in object_list:
        f.write(i)
        f.write('\n')

start convert pascal voc format into coco format.

In [11]:
# convert for training set
!python voc2coco.py --ann_dir sample/Annotations \
    --ann_ids sample/dataset_ids/train.txt \
    --labels sample/labels.txt \
    --output sample/instances_train.json \
    --ext xml

Start converting !
100% 10275/10275 [00:00<00:00, 11892.45it/s]


In [12]:
# convert for validation set 
!python voc2coco.py --ann_dir sample/Annotations \
    --ann_ids sample/dataset_ids/val.txt \
    --labels sample/labels.txt \
    --output sample/instances_val.json \
    --ext xml

Start converting !
100% 3425/3425 [00:00<00:00, 12110.61it/s]


In [13]:
# convert for test set
!python voc2coco.py --ann_dir sample/Annotations \
    --ann_ids sample/dataset_ids/test.txt \
    --labels sample/labels.txt \
    --output sample/instances_test.json \
    --ext xml

Start converting !
100% 3425/3425 [00:00<00:00, 13046.12it/s]


we done convert format.

# III. Clone tool for training model and processing for right format.

See detail: https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch

In [14]:
os.chdir('/content')

if "projects" not in os.getcwd():
  !git clone --depth 1 https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch
  os.chdir('/content/Yet-Another-EfficientDet-Pytorch')
  sys.path.append('.')
else:
  !git pull

Cloning into 'Yet-Another-EfficientDet-Pytorch'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 47 (delta 3), reused 24 (delta 1), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [15]:
!mkdir /content/Yet-Another-EfficientDet-Pytorch/datasets
!mkdir /content/Yet-Another-EfficientDet-Pytorch/datasets/voc
!mkdir /content/Yet-Another-EfficientDet-Pytorch/datasets/voc/train
!mkdir /content/Yet-Another-EfficientDet-Pytorch/datasets/voc/val
!mkdir /content/Yet-Another-EfficientDet-Pytorch/datasets/voc/test   
!mkdir /content/Yet-Another-EfficientDet-Pytorch/datasets/voc/annotations

move the file to the right location

In [16]:
path = '/content/voc2coco/dataset/train/JPEGImages'
for filename in os.listdir(path):
    fullname = os.path.join(path,filename)
    shutil.move(fullname, "/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/train") 

path = '/content/voc2coco/dataset/val/JPEGImages'
for filename in os.listdir(path):
    fullname = os.path.join(path,filename)
    shutil.move(fullname, "/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/val") 

path = '/content/voc2coco/dataset/test/JPEGImages'
for filename in os.listdir(path):
    fullname = os.path.join(path,filename)
    shutil.move(fullname, "/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/test") 

path1 = '/content/voc2coco/sample/instances_train.json'
path2 = '/content/voc2coco/sample/instances_val.json'
path3 = '/content/voc2coco/sample/instances_test.json'

shutil.move(path1, '/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/annotations')
shutil.move(path2, '/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/annotations')
shutil.move(path3, '/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/annotations')

'/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/annotations/instances_test.json'

because the type of id is string, so we need to convert it into integer.

In [17]:
def convert_id(path):
  f = open(path)
  data = json.load(f)

  for i in range(len(data['images'])):
      value = data['images'][i]['id']
      value1 = value[:4] + value[5:]
      data['images'][i]['id'] = int(value1)
  
  for i in range(len(data['annotations'])):
      value = data['annotations'][i]['image_id']
      value1 = value[:4] + value[5:]
      data['annotations'][i]['image_id'] = int(value1)

  !rm -rf path
  with open(path, 'w') as f:
    f.write(json.dumps(data))

In [18]:
# for traning, validation and test set:
convert_id('/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/annotations/instances_train.json')
convert_id('/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/annotations/instances_val.json')
convert_id('/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/annotations/instances_test.json')

we have voc.yml file: this file have some parameter.

In [19]:
shutil.copy("/content/gdrive/MyDrive/Colab Notebooks/voc.yml", "/content/Yet-Another-EfficientDet-Pytorch/projects")

'/content/Yet-Another-EfficientDet-Pytorch/projects/voc.yml'

In [20]:
#showing its contents here
!cat projects/voc.yml 

project_name: voc  # also the folder name of the dataset that under data_path folder
train_set: train
val_set: val
num_gpus: 1

# mean and std in RGB order, actually this part should remain unchanged as long as your dataset is similar to coco.
mean: [ 0.485, 0.456, 0.406 ]
std: [ 0.229, 0.224, 0.225 ]

# this anchor is adapted to the dataset
anchors_scales: '[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]'
anchors_ratios: '[(1.0, 1.0), (1.3, 0.8), (1.9, 0.5)]'

obj_list: [ 'aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

# IV.Training: use pre-trained model in COCO dataset

Continue training for 15 latter epochs with lr = 1e-3

In [21]:
# download last checkpoint
! mkdir weights
shutil.copy("/content/gdrive/MyDrive/Colab Notebooks/efficientdet-d0.pth", "/content/Yet-Another-EfficientDet-Pytorch/weights")

'/content/Yet-Another-EfficientDet-Pytorch/weights/efficientdet-d0.pth'

training

In [22]:
!python train.py -c 0 -p voc --head_only True \
                   --lr 1e-3 --batch_size 32 \
                   --load_weights /content/Yet-Another-EfficientDet-Pytorch/weights/efficientdet-d0.pth  --num_epochs 15 --save_interval 300


loading annotations into memory...
Done (t=0.12s)
creating index...
index created!
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
[Info] loaded weights: efficientdet-d0.pth, resuming checkpoint from step: 0
[Info] freezed backbone
Step: 299. Epoch: 0/15. Iteration: 300/321. Cls loss: 0.25593. Reg loss: 0.88537. Total loss: 1.14130:  93% 299/321 [04:40<00:18,  1.22it/s]checkpoint...
Step: 320. Epoch: 0/15. Iteration: 321/321. Cls loss: 0.31762. Reg loss: 1.00795. Total loss: 1.32557: 1

we found that the : loss of validation set changes little, so we stop training here.

# V. Evaluation on validation set to find best checkpoint.

we just evaluate checkpoints that the loss of validation set < 1.23

In [24]:
checkpoint = ['efficientdet-d0_5_1926.pth', 'efficientdet-d0_8_2700.pth', 'efficientdet-d0_8_2889.pth',\
              'efficientdet-d0_9_3000.pth', 'efficientdet-d0_13_4200.pth', 'efficientdet-d0_13_4494.pth',\
              'efficientdet-d0_14_4800.pth']
for filename in checkpoint:
  if filename != 'tensorboard':
    print('-------------------------------------------------------\n\n\n')
    print(filename)

    weight_file = filename
    !python coco_eval.py -c 0 -p voc -w "logs/voc/{weight_file}"

-------------------------------------------------------



efficientdet-d0_5_1926.pth
running coco-style evaluation on project voc, weights logs/voc/efficientdet-d0_5_1926.pth...
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
100% 3425/3425 [03:26<00:00, 16.55it/s]
Loading and preparing results...
DONE (t=0.56s)
creating index...
index created!
BBox
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.90s).
Accumulating evaluation results...
DONE (t=1.70s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.453
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.648
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.503
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.120
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.305
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.551
 

you can see that: efficientdet-d0_14_4800.pth checkpoint has highest scores.

# VI. Evaluation on test set:

we need to fix format to use EfficientDet tool

In [25]:
!rm -rf /content/Yet-Another-EfficientDet-Pytorch/datasets/voc/val
!mkdir /content/Yet-Another-EfficientDet-Pytorch/datasets/voc/val

path = '/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/test'
for filename in os.listdir(path):
    fullname = os.path.join(path,filename)
    shutil.move(fullname, "/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/val") 

In [26]:
!rm -rf /content/Yet-Another-EfficientDet-Pytorch/datasets/voc/annotations/instances_val.json

old_name = '/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/annotations/instances_test.json'
new_name = '/content/Yet-Another-EfficientDet-Pytorch/datasets/voc/annotations/instances_val.json'
os.rename(old_name, new_name)

In [27]:
# evaluate on test set

weight_file = 'efficientdet-d0_14_4800.pth'
!python coco_eval.py -c 0 -p voc -w "logs/voc/{weight_file}"

running coco-style evaluation on project voc, weights logs/voc/efficientdet-d0_14_4800.pth...
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
100% 3425/3425 [03:24<00:00, 16.78it/s]
Loading and preparing results...
DONE (t=0.55s)
creating index...
index created!
BBox
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=6.43s).
Accumulating evaluation results...
DONE (t=1.56s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.465
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.664
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.507
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.115
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.328
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.558
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.446
 Avera